In [10]:
# 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import os
import joblib

from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from catboost import CatBoostRegressor

In [11]:
path = os.getcwd() + "/data/data.csv"
df = pd.read_csv(path)
df[['gu', 'ro']] = df['Address'].str.split(' ', expand=True).iloc[:, :2]
df['Supply_type'] = df['Supply_type'].str.replace(r'\D', '', regex=True)
qty = (3 - df['Cutline_rate']) * 10 + df['Cutline_score']

df.drop(
    columns=[
        'Name','Address', 'Latitude', 'Longitude', 'Infra_score',
        'Gender','Shared','Quarter','Counts_supermarket','Counts_laundry',
        'Counts_pharmacy','Cutline_rate','Cutline_score'
    ],
    inplace=True
)

df = pd.get_dummies(data=df)
df['Qty'] = qty

In [12]:
X = df.drop(columns='Qty')
y = df['Qty']

oversampler = RandomOverSampler(sampling_strategy={2: 500, 3: 500})
X, y = oversampler.fit_resample(X, y)

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2
)

model = CatBoostRegressor(
    iterations=1000,
    depth=6,
    learning_rate=0.1,
    loss_function='RMSE',
    verbose=100,
    task_type='CPU'
)

model.fit(
    X, y,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=10,
    use_best_model=True,
    plot=True
)

y_pred = model.predict(X_valid)

# print('Rate : ')
# print(classification_report(y_valid,y_pred))
# print('Rate CM:')
# print(confusion_matrix(y_valid,y_pred))
print('Score MAE:',mean_absolute_error(y_valid,y_pred))
print('Score R2:',r2_score(y_valid,y_pred))

joblib.dump(model, os.getcwd() + '/main.pkl')


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 9.7543095	test: 9.6436889	best: 9.6436889 (0)	total: 1.05ms	remaining: 1.05s
100:	learn: 2.2553017	test: 2.1839880	best: 2.1839880 (100)	total: 110ms	remaining: 975ms
200:	learn: 1.8713646	test: 1.8521514	best: 1.8521514 (200)	total: 202ms	remaining: 804ms
300:	learn: 1.6253360	test: 1.6361559	best: 1.6361559 (300)	total: 295ms	remaining: 685ms
400:	learn: 1.4522892	test: 1.4783318	best: 1.4783318 (400)	total: 389ms	remaining: 581ms
500:	learn: 1.3337121	test: 1.3600957	best: 1.3600957 (500)	total: 484ms	remaining: 482ms
600:	learn: 1.2154165	test: 1.2339940	best: 1.2339940 (600)	total: 578ms	remaining: 384ms
700:	learn: 1.1126003	test: 1.1370969	best: 1.1370969 (700)	total: 669ms	remaining: 285ms
800:	learn: 1.0182635	test: 1.0370160	best: 1.0370160 (800)	total: 761ms	remaining: 189ms
900:	learn: 0.9371871	test: 0.9513802	best: 0.9513802 (900)	total: 852ms	remaining: 93.6ms
999:	learn: 0.8622657	test: 0.8791159	best: 0.8791159 (999)	total: 951ms	remaining: 0us

bestTest = 0.

['/home/chansa/aivle/Projects/bigpj/main.pkl']

In [15]:
joblib.load(os.getcwd() + '/main.pkl')
y_pred = model.predict(X_valid)
print(y_pred)
df = pd.read_csv(path)
df['Qty'] = y_pred
df.to_csv(os.getcwd() + '/data/data_catboost.csv')

[ 2.15469516  2.15469516 24.4048619  28.00728158 28.05301542  3.02657945
  3.02657945  3.0320101   2.15469516 23.41673542  3.0320101  20.85952492
 23.09514963  3.0320101  25.6746261  18.92516255  3.02657945 18.08241161
  3.02657945  3.02657945 20.31714428  2.04161223  3.02657945 23.51599202
  3.02657945  1.95665632  2.10946143 27.75793294  3.02657945 22.84564895
 25.74459163  3.0320101   2.28967487 14.43638875 27.70275363 25.31033173
  3.02657945 18.0090579  27.54236287 24.03531608  2.02199696  1.94526816
  3.02657945 26.33156605  1.97667845  3.0320101   2.02852266  1.95665632
 26.52393372 24.55292246 27.7419897   3.0320101   3.02657945  3.02657945
  1.97667845 17.01819721  7.08952209 25.28323879  2.02852266  7.11827239
  3.02657945  3.0320101   3.02657945 14.43341941  1.94526816 24.84072251
  7.00362159 16.32267713  3.02657945  2.08626082  3.0320101   3.02657945
  2.08626082  2.10946143 29.27972992 14.82024676  2.09616081  3.02657945
  3.0320101   1.95665632  3.0320101  18.62128516  2

ValueError: Length of values (372) does not match length of index (876)